## This notebook classifies movie reviews as positive or negative using the text of the review. This is an example of binary—or two-class—classification, an important and widely applicable kind of machine learning problem.

## The tutorial demonstrates the basic application of transfer learning with TensorFlow Hub and Keras.

## We'll use the IMDB dataset that contains the text of 50,000 movie reviews from the Internet Movie Database. These are split into 25,000 reviews for training and 25,000 reviews for testing. The training and testing sets are balanced, meaning they contain an equal number of positive and negative reviews.

## This notebook uses tf.keras, a high-level API to build and train models in TensorFlow, and TensorFlow Hub, a library and platform for transfer learning. For a more advanced text classification tutorial using tf.keras

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print('GPU is ','available' if tf.config.experimental.list_physical_devices('GPU') else 'NOT AVAILALBLE')

Version:  2.1.0
Eager mode:  True
Hub version:  0.9.0
GPU is  NOT AVAILALBLE


In [15]:
 pip install IProgress

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\python\python37\python.exe -m pip install --upgrade pip' command.


## The IMDB dataset is available on imdb reviews or on TensorFlow datasets.

In [2]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.

train_data,validation_data,test_data = tfds.load(name = 'imdb_reviews', split = ('train[:60%]','train[60%:]', 'test'),
                                                as_supervised = True)

Shuffling and writing examples to C:\Users\Daraqshan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteVV5Z0P\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\Daraqshan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteVV5Z0P\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\Daraqshan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteVV5Z0P\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\Daraqshan\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


## The label is an integer value of either 0 or 1, where 0 is a negative review, and 1 is a positive review.

In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [4]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

## Build the model
### The neural network is created by stacking layers—this requires three main architectural decisions:

### How to represent the text?
### How many layers to use in the model?
### How many hidden units to use for each layer?

### One way to represent the text is to convert sentences into embeddings vectors. We can use a pre-trained text embedding as the first layer, which will have three advantages:

### we don't have to worry about text preprocessing,
### we can benefit from transfer learning,
### the embedding has a fixed size, so it's simpler to process.

### Output shape of the embeddings is: (num_examples, embedding_dimension)

In [5]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


### Since this is a binary classification problem and the model outputs logits (a single-unit layer with a linear activation), we'll use the binary_crossentropy loss function.

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 8s 265ms/step - loss: 0.8455 - accuracy: 0.5656 - val_loss: 0.7298 - val_accuracy: 0.5939
Epoch 2/20
30/30 [==============================] - 3s 98ms/step - loss: 0.6809 - accuracy: 0.6192 - val_loss: 0.6547 - val_accuracy: 0.6342
Epoch 3/20
30/30 [==============================] - 3s 95ms/step - loss: 0.6185 - accuracy: 0.6581 - val_loss: 0.6115 - val_accuracy: 0.6564
Epoch 4/20
30/30 [==============================] - 3s 95ms/step - loss: 0.5767 - accuracy: 0.6880 - val_loss: 0.5758 - val_accuracy: 0.6852
Epoch 5/20
30/30 [==============================] - 3s 115ms/step - loss: 0.5397 - accuracy: 0.7127 - val_loss: 0.5430 - val_accuracy: 0.7034
Epoch 6/20
30/30 [==============================] - 3s 102ms/step - loss: 0.5045 - accuracy: 0.7403 - val_loss: 0.5128 - val_accuracy: 0.7386
Epoch 7/20
30/30 [==============================] - 3s 98ms/step - loss: 0.4714 - accuracy: 0.7648 - val_loss: 0.4840 - val_accuracy: 0.7519
Epoch 8/20

In [9]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.325
accuracy: 0.852


## as_supervised: bool, if True, the returned tf.data.Dataset will have a 2-tuple structure (input, label) according to builder.info.supervised_keys.

## The batch size is a number of samples processed before the model is updated. The number of epochs is the number of complete passes through the training dataset. The size of a batch must be more than or equal to one and less than or equal to the number of samples in the training dataset.